In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as snsb

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
songs = pd.read_csv('songdata.csv')

In [4]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
#resample only 5000 songs
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

#replace \n in the text
songs['text'] = songs['text'].str.replace(r'\n', '')

#intialize the tf-idf vector
tfidf = TfidfVectorizer(analyzer = 'word', stop_words='english')

#fit and transform tfidf

#The fit() method calculates the various required parameters, and the transform() method applies
#the calculated parameters to standardize the data. In the case of the StandardScaler(), 
#the mean and the standard deviation are calculated and the data is scaled and centered to have 
#a mean of 0 and a standard deviation of 1 and it is a method from preprocessing class in sklearn

lyrics_matrix = tfidf.fit_transform(songs['text'])

C:\Users\prane\AppData\Local\Temp\ipykernel_6944\2750878308.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n', '')


In [6]:
cosine_similarities = cosine_similarity(lyrics_matrix)

# creating a dictionary
similarities = {}

#50 most similar songs to each song in our data set
for i in range(len(cosine_similarities)):
    
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs.
    similar_indices = cosine_similarities[i].argsort()[:-50:-1]
    
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    #iloc will retrieve rows
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], 
                                           songs['artist'][x]) for x in similar_indices][1:]


In [18]:
# create a class ContentBased Recommender
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix
        
    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number{i+1}: ")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score")
            print("--------------------")
            
    def recommend(self, recommendation):
        #Get song to find recommendations for
        song = recommendation['song']
        #Get number of songs to recommend
        number_songs = recommendation['number_songs']
        #Get number of songs most similar from the matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        #print each item
        self._print_message(song = song, recom_song = recom_song)
              

In [19]:
#Now, instantiate the class

recommendations = ContentBasedRecommender(similarities)

In [20]:
#Pick a song from the data set and make a recommendation
recommendation = {
    "song":songs['song'].iloc[10],
    "number_songs":4
}

In [21]:
recommendations.recommend(recommendation)

The 4 recommended songs for Exactly Like You are:
Number1: 
Someone Like You by John Waite with 0.279 similarity score
--------------------
Number2: 
Love Is Just Around The Corner -(With Take 6) by Kenny Rogers with 0.245 similarity score
--------------------
Number3: 
If I Knew What I Know Now by Kenny Rogers with 0.221 similarity score
--------------------
Number4: 
I Want To Know What Love Is by Mariah Carey with 0.215 similarity score
--------------------


In [22]:
#pick up another random song

recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 6
}

In [23]:
recommendations.recommend(recommendation2)

The 6 recommended songs for Evening In June are:
Number1: 
I'll Be On My Way by The Beatles with 0.339 similarity score
--------------------
Number2: 
Evening Star by Judas Priest with 0.322 similarity score
--------------------
Number3: 
A Lovely Way To Spend An Evening by Engelbert Humperdinck with 0.293 similarity score
--------------------
Number4: 
In The Valley (Where The Evening Sun Goes Down) by Judy Garland with 0.267 similarity score
--------------------
Number5: 
Cliches by Jimmy Buffett with 0.149 similarity score
--------------------
Number6: 
Everyone's Gone To The Moon by Nina Simone with 0.142 similarity score
--------------------
